In [ ]:
import re
from transformers import AutoTokenizer

# 初始化tokenizer
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct')

from utils import TokenTemplate
             

# 示例使用
TEMPLATE = """Here is a problem, a section of a article that may contain the answer to the problem and a previous memory. Please read carefully and update the memory based on given section to help answer the problem. 
You should keep the relevant information in the memory while adding new information.

<problem> 
{problem}
</problem>

<memory>
{memory}
</memory>

<section>
{section}
</section>

Updated memory (should be enclosed in <memory> and </memory>)
<memory>
"""

processor = TokenTemplate(TEMPLATE)
processor.init(tokenizer)

# 假设传入的 token ids（在实际使用时应该从模型或其他地方获取）
kwarg_text = dict(
    problem="What is the capital of France?",
    section="Here is a introduction to France. France is a country in Western Europe. Its capital is Paris.",
    memory="No previous memory",
)
kwargs_token_ids = {
    k: tokenizer.encode(v, add_special_tokens=False) for k, v in kwarg_text.items()
}
# 格式化模板
formatted_template = processor.format(**kwargs_token_ids)
print(tokenizer.decode(formatted_template))
print(TEMPLATE.format(**kwarg_text) == tokenizer.decode(formatted_template))


In [ ]:
tokenizer.__dict__

In [ ]:
message = tokenizer.apply_chat_template([{'role':'user','content':'{message}'}],add_generation_prompt=True, tokenize=False)

In [ ]:
def find_substrings(lst, startlist, endlist):
    start_len = len(startlist)
    end_len = len(endlist)
    substrings = []
    
    i = 0
    while i <= len(lst) - start_len - end_len:
        # 找到开始部分
        if lst[i:i+start_len] == startlist:
            j = i + start_len
            # 找到结束部分
            while j <= len(lst) - end_len and lst[j:j+end_len] != endlist:
                j += 1
            if j <= len(lst) - end_len and lst[j:j+end_len] == endlist:
                substrings.append(lst[i+start_len:j])
        i += 1
    
    return substrings

# 示例用法
lst = [1, 2, 3, 4, 5]
startlist = [1, 2] # <summary>
endlist = [4, 5] # </summary>
print(find_substrings(lst, startlist, endlist))
